# Topic modelling Q&D pilot
https://medium.datadriveninvestor.com/nlp-with-lda-analyzing-topics-in-the-enron-email-dataset-20326b7ae36f
https://github.com/shoreason/enron-topic-modeling/blob/master/enron_lda.ipynb


In [1]:
import pandas as pd


In [98]:
# Create a small set of emails (10 000)
"""emails_read_all = pd.read_csv('./csv/emails_df.csv')
enron = emails_read_all.head[:10000]
enron.to_csv('./csv/emails_df_10000.csv', index=False)"""

"emails_read_all = pd.read_csv('./csv/emails_df.csv')\nenron = emails_read_all.head[:10000]\nenron.to_csv('./csv/emails_df_10000.csv', index=False)"

In [100]:
#enron = pd.read_csv('./csv/emails_df_10000.csv')
nron = pd.read_csv('./csv/emails_df.csv')
#enron = enron[:1000]
#enron = pd.read_csv('./csv/emails_df.csv')

In [4]:
enron.columns


Index(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'content', 'user'],
      dtype='object')

In [96]:
enron.drop_duplicates(subset=['content'], inplace=True)


In [97]:
enron.shape

(653, 18)

In [5]:
enron.shape #10 000 rows, 18 columns
print(enron.columns)
enron.rename(columns={'content': 'body'}, inplace=True)


Index(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'content', 'user'],
      dtype='object')


In [6]:
enron = enron[['From', 'To', 'Subject', 'body']]
print(enron.head())



                                     From  \
0  frozenset({'phillip.allen@enron.com'})   
1  frozenset({'phillip.allen@enron.com'})   
2  frozenset({'phillip.allen@enron.com'})   
3  frozenset({'phillip.allen@enron.com'})   
4  frozenset({'phillip.allen@enron.com'})   

                                       To    Subject  \
0     frozenset({'tim.belden@enron.com'})        NaN   
1  frozenset({'john.lavorato@enron.com'})        Re:   
2   frozenset({'leah.arsdall@enron.com'})   Re: test   
3    frozenset({'randall.gay@enron.com'})        NaN   
4     frozenset({'greg.piper@enron.com'})  Re: Hello   

                                                body  
0                          Here is our forecast\n\n   
1  Traveling to have a business meeting takes the...  
2                     test successful.  way to go!!!  
3  Randy,\n\n Can you send me a schedule of the s...  
4                Let's shoot for Tuesday at 11:45.    


In [7]:
import re
import numpy as np
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy # for lemmatization
# for plotting
import pyLDAvis
import pyLDAvis.gensim_models  # the module 'gensim' has renamed to gensim_models
import matplotlib.pyplot as plt

/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [8]:
# prep NLTK stop words
from nltk.corpus import stopwords

# You have to install the word on your local machine , 3.84 GB!!
# https://www.nltk.org/data.html
stop_words = stopwords.words('english') # will cause error if not installed accordint to the instructions above
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'send', 'com']) # if replied the mail contains thes words -> remove them



In [9]:
print(enron.iloc[1]['body']) # displays info below

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  

My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.



In [10]:
# Convert email body to list
#enron = enron[:1000]
data = enron['body'].values.tolist()
subject = enron['Subject'].values.tolist()


In [11]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [12]:
data_words = list(sent_to_words(data))

In [13]:
print(data_words[1])

['traveling', 'to', 'have', 'business', 'meeting', 'takes', 'the', 'fun', 'out', 'of', 'the', 'trip', 'especially', 'if', 'you', 'have', 'to', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'the', 'business', 'plan', 'meetings', 'here', 'then', 'take', 'trip', 'without', 'any', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'and', 'get', 'some', 'honest', 'opinions', 'on', 'whether', 'trip', 'is', 'even', 'desired', 'or', 'necessary', 'as', 'far', 'as', 'the', 'business', 'meetings', 'think', 'it', 'would', 'be', 'more', 'productive', 'to', 'try', 'and', 'stimulate', 'discussions', 'across', 'the', 'different', 'groups', 'about', 'what', 'is', 'working', 'and', 'what', 'is', 'not', 'too', 'often', 'the', 'presenter', 'speaks', 'and', 'the', 'others', 'are', 'quiet', 'just', 'waiting', 'for', 'their', 'turn', 'the', 'meetings', 'might', 'be', 'better', 'if', 'held', 'in', 'round', 'table', 'discussion', 'format', 'my', 'suggestion', 'for', 'where', 'to', 'go', 'is'

In [14]:
from gensim.models.phrases import Phrases, Phraser

In [15]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

In [16]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [17]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[200]]])

['beth', 'here', 'are', 'our', 'addresses', 'for', 'dsl', 'lines', 'hunter_shively', 'gawain', 'houston_tx', 'phillip', 'allen', 'merlin_ct', 'houston_tx', 'mike_grigsby', 'meadow', 'lake', 'houston_tx', 'thanks', 'phillip']


In [18]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words 
data_words_nostops = remove_stopwords(data_words)

In [20]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

#python -m spacy download en

In [21]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#nlp = spacy.load('en', disable=['parser', 'ner']) # this is how it used to be
nlp = spacy.load('en_core_web_sm')


In [22]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[1])

['travel', 'business', 'meeting', 'take', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'suggest', 'hold', 'business', 'plan', 'meeting', 'take', 'trip', 'formal', 'business', 'meeting', 'even', 'try', 'get', 'honest', 'opinion', 'trip', 'even', 'desire', 'necessary', 'far', 'business', 'meeting', 'think', 'productive', 'try', 'stimulate', 'discussion', 'different', 'group', 'work', 'often', 'presenter', 'speak', 'other', 'quiet', 'wait', 'turn', 'meeting', 'well', 'hold', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'play', 'golf', 'rent', 'boat', 'jet', 'ski', 'fly', 'somewhere', 'take', 'much', 'time']


In [23]:
#emails_lemmatized = pd.DataFrame(data_lemmatized, columns=['lemmatized_subject'])
len(data_lemmatized)
new_column = pd.Series(data_lemmatized)
enron['body_lem'] = new_column

In [24]:
print(data_lemmatized[1])

['travel', 'business', 'meeting', 'take', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'suggest', 'hold', 'business', 'plan', 'meeting', 'take', 'trip', 'formal', 'business', 'meeting', 'even', 'try', 'get', 'honest', 'opinion', 'trip', 'even', 'desire', 'necessary', 'far', 'business', 'meeting', 'think', 'productive', 'try', 'stimulate', 'discussion', 'different', 'group', 'work', 'often', 'presenter', 'speak', 'other', 'quiet', 'wait', 'turn', 'meeting', 'well', 'hold', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'play', 'golf', 'rent', 'boat', 'jet', 'ski', 'fly', 'somewhere', 'take', 'much', 'time']


In [25]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [26]:
mallet_path = './mallet-2.0.8/bin/mallet'

In [27]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [28]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [29]:
print((lda_model))

LdaModel(num_terms=4438, num_topics=14, decay=0.5, chunksize=100)


In [30]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [31]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(0, '0.035*"phillip" + 0.030*"get" + 0.020*"email" + 0.020*"new" + 0.018*"mail" + 0.018*"thank" + 0.018*"receive" + 0.017*"file" + 0.015*"live" + 0.015*"follow"'), (1, '0.032*"go" + 0.031*"need" + 0.029*"work" + 0.025*"check" + 0.021*"time" + 0.020*"phillip" + 0.020*"comment" + 0.016*"call" + 0.016*"see" + 0.015*"let"'), (2, '0.058*"request" + 0.042*"phillip" + 0.033*"gas" + 0.029*"need" + 0.026*"forward" + 0.025*"basis" + 0.022*"change" + 0.020*"trading" + 0.020*"point" + 0.019*"pm"'), (3, '0.049*"image_image" + 0.046*"buy" + 0.035*"socal" + 0.025*"physical" + 0.020*"downgrade" + 0.018*"index" + 0.017*"daily" + 0.017*"trade" + 0.017*"image" + 0.015*"market"'), (4, '0.031*"power" + 0.019*"say" + 0.018*"price" + 0.017*"gas" + 0.016*"utility" + 0.015*"market" + 0.014*"iso" + 0.014*"plant" + 0.012*"report" + 0.012*"line"'), (5, '0.046*"bid" + 0.039*"do" + 0.034*"pay" + 0.030*"spread" + 0.029*"ready" + 0.024*"deposit" + 0.023*"model" + 0.018*"way" + 0.018*"suggest" + 0.015*"print"'), (6, 

In [32]:
doc_lda = lda_model[corpus]

In [33]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.576028460689164


In [34]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3601152398346038


In [35]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/jari/DATA/Projects/Clustering-Enron_UsedCase/enron/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of impor

In [36]:
print(lda_model.show_topic(0))

[('phillip', 0.035193995), ('get', 0.030468876), ('email', 0.020496476), ('new', 0.019649263), ('mail', 0.018474707), ('thank', 0.018151915), ('receive', 0.017657977), ('file', 0.016536191), ('live', 0.014699136), ('follow', 0.014656359)]


In [37]:
pyLDAvis.display(vis)

## How to get top emails per topic?
There are actually two cases:
<br> a. Best email to given topic
<br> b. Dominant topic for a given email


In [70]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()


    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        #print("i: ", i)
        #print("row: ", row)
        #print("row[1]: ", row[1])
        row = sorted(
            row[0], key=lambda x: (x[1]), 
            reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [64]:
print(type(lda_model[corpus]))
print(len(lda_model[corpus]))
print("corpus type:", type(corpus))
print("corpus len", len(corpus))

<class 'gensim.interfaces.TransformedCorpus'>
1000
corpus type: <class 'list'>
corpus len 1000


In [71]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)



/var/folders/6r/0sxxkqln2g73tp73l2p7xx200000gn/T/ipykernel_38332/825188672.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
/var/folders/6r/0sxxkqln2g73tp73l2p7xx200000gn/T/ipykernel_38332/825188672.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [72]:

df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,4,0.3755,"power, say, price, gas, utility, market, iso, ...",Here is our forecast\n\n
1,11,0.8573,"ee, enronxgate, position, meeting, issue, ect,...",Traveling to have a business meeting takes the...
2,1,0.5054,"go, need, work, check, time, phillip, comment,...",test successful. way to go!!!
3,9,0.2948,"email, phillip, information, need, list, thank...","Randy,\n\n Can you send me a schedule of the s..."
4,1,0.3406,"go, need, work, check, time, phillip, comment,...",Let's shoot for Tuesday at 11:45.


In [73]:
df_topic_sents_keywords['Dominant_Topic'].value_counts()

0     252
7     138
1     110
4      91
10     78
6      71
2      70
9      48
3      41
11     39
13     28
8      23
5      11
Name: Dominant_Topic, dtype: int64

In [90]:
topic_9 = df_topic_sents_keywords[df_topic_sents_keywords['Dominant_Topic'] == 1].sort_values(by=['Perc_Contribution'], ascending=False)
topic_9.head()
topic_9.rename(columns={0: 'Text'}, inplace=True)
topic_9.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
674,1,0.9028,"go, need, work, check, time, phillip, comment,...","Brad,\n\n With regard to Tori Kuykendall, I wo..."
86,1,0.9028,"go, need, work, check, time, phillip, comment,...","Brad,\n\n With regard to Tori Kuykendall, I wo..."
32,1,0.8889,"go, need, work, check, time, phillip, comment,...",---------------------- Forwarded by Phillip K ...
621,1,0.8889,"go, need, work, check, time, phillip, comment,...",---------------------- Forwarded by Phillip K ...
248,1,0.7577,"go, need, work, check, time, phillip, comment,...",I just spoke to the insurance company. They a...


In [92]:
messages = topic_9[0:10]['Text'].to_list()
print(type(messages))
for msg in messages:
    print(msg)




<class 'list'>
Brad,

 With regard to Tori Kuykendall, I would like to promote her to commercial 
manager instead of converting her from a commercial support manager to an 
associate.  Her duties since the beginning of the year have been those of a 
commercial manager.  I have no doubt that she will compare favorably to 
others in that category at year end.  

 Martin Cuilla on the central desk is in a similiar situation as Tori.  
Hunter would like Martin handled the same as Tori.

 Let me know if there are any issues.

Phillip
Brad,

 With regard to Tori Kuykendall, I would like to promote her to commercial 
manager instead of converting her from a commercial support manager to an 
associate.  Her duties since the beginning of the year have been those of a 
commercial manager.  I have no doubt that she will compare favorably to 
others in that category at year end.  

 Martin Cuilla on the central desk is in a similiar situation as Tori.  
Hunter would like Martin handled the same as